In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

path = 'NVDA.csv'

In [2]:
df = pd.read_csv(path, encoding='latin-1') # or 'cp1252'
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1999-01-22 00:00:00-05:00,0.040122,0.044779,0.035585,0.037615,2714688000,0.0,0.0
1,1999-01-25 00:00:00-05:00,0.040600,0.042033,0.037615,0.041556,510480000,0.0,0.0
2,1999-01-26 00:00:00-05:00,0.042033,0.042869,0.037734,0.038331,343200000,0.0,0.0
3,1999-01-27 00:00:00-05:00,0.038451,0.039406,0.036301,0.038212,244368000,0.0,0.0
4,1999-01-28 00:00:00-05:00,0.038212,0.038451,0.037854,0.038092,227520000,0.0,0.0


Descrizione tabella:
- Date: La data del record (formato timestamp).

- Open, High, Low, Close: Prezzi di apertura, massimo, minimo e chiusura.
- Volume: Quantità di azioni scambiate.
- Dividends: Dividendi distribuiti (spesso zero).
- Stock Splits: Eventuali frazionamenti azionari.


2) Analisi Esplorativa dei dati (EDA)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6513 entries, 0 to 6512
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          6513 non-null   object 
 1   Open          6513 non-null   float64
 2   High          6513 non-null   float64
 3   Low           6513 non-null   float64
 4   Close         6513 non-null   float64
 5   Volume        6513 non-null   int64  
 6   Dividends     6513 non-null   float64
 7   Stock Splits  6513 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 407.2+ KB


In [4]:
df.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,6513.000000,6513.000000,6513.000000,6513.000000,6.513000e+03,6513.000000,6513.000000
mean,7.902262,8.046122,7.745036,7.902351,6.015231e+08,0.000028,0.003301
std,21.597247,21.979239,21.151316,21.581519,4.311052e+08,0.000359,0.141388
min,0.032003,0.032600,0.030569,0.031286,1.968000e+07,0.000000,0.000000
25%,0.256325,0.263662,0.249906,0.256325,3.410440e+08,0.000000,0.000000
50%,0.434869,0.441264,0.427963,0.434928,5.032080e+08,0.000000,0.000000
75%,4.429733,4.493741,4.360737,4.440177,7.326600e+08,0.000000,0.000000
max,149.339720,152.879470,146.249911,148.869751,9.230856e+09,0.010000,10.000000


3) Visualizzazione

In [5]:
# Using plotly.express
import plotly.express as px

#df = px.data.stocks()
fig = px.line(df, x='Date', y="Close") #metti nelle y una media tra prezzo più basso e più alto della giornata
fig.show()